# Core Base Classes

> Base classes and types for all daisyUI components

In [ ]:
#| default_exp core.base

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, Optional, List, Union, Literal
from dataclasses import dataclass, field
from enum import Enum
from fasthtml.common import *
from cjm_tailwind_utils.all import TailwindBuilder
from cjm_fasthtml_daisyui.core.types import (
    # Enums
    DaisyPosition,
    DaisyBreakpoint,
    DaisySize,
    SemanticColor,
    ColorUtility,
    DaisyComponentType,
    # Protocols
    CSSContributor,
    ComponentProtocol,
    FeatureSupport,
    # Utility functions
    ensure_list,
    ensure_dict
)
from cjm_fasthtml_daisyui.core.colors import (
    ColorBuilder, ColorMixin,
    apply_semantic_colors, get_color_classes
)

In [ ]:
#| export
def deduplicate_classes(
    *class_sources: Union[str, List[str], None] # Multiple sources of CSS classes (strings, lists, or None)
) -> str:  # Space-separated string of deduplicated CSS classes, sorted alphabetically
    """Deduplicate CSS classes from multiple sources.
    
    Takes multiple sources of CSS classes (strings, lists, or None values) and
    returns a single space-separated string with duplicates removed and sorted
    alphabetically for consistency.
    
    Args:
        *class_sources: Variable number of class sources, each can be:
            - A space-separated string of CSS classes
            - A list of CSS class strings
            - None (will be ignored)
    
    Returns:
        A space-separated string of unique CSS classes, sorted alphabetically
        
    Examples:
        >>> deduplicate_classes("btn btn-primary", ["btn", "btn-lg"])
        'btn btn-lg btn-primary'
        
        >>> deduplicate_classes("hidden md:block", None, ["hidden", "lg:block"])
        'hidden lg:block md:block'
    """
    all_classes = set()
    
    for source in class_sources:
        if source is None:
            continue
        elif isinstance(source, str):
            # Split space-separated string into individual classes
            all_classes.update(source.split())
        elif isinstance(source, list):
            # Add all items from the list
            all_classes.update(source)
        else:
            # For any other iterable, try to add its items
            try:
                all_classes.update(source)
            except TypeError:
                # If it's not iterable, convert to string and split
                all_classes.update(str(source).split())
    
    # Return sorted for consistent output
    return " ".join(sorted(all_classes))

In [ ]:
# Testing the deduplicate_classes function
print("Testing deduplicate_classes function:")
print("="*50)

# Test with duplicate classes
result1 = deduplicate_classes("btn btn-primary", ["btn", "btn-lg"])
print(f"Test 1 - Duplicates: {result1}")

# Test with None values
result2 = deduplicate_classes("hidden md:block", None, ["hidden", "lg:block"])
print(f"Test 2 - With None: {result2}")

# Test with multiple string sources
result3 = deduplicate_classes(
    "card card-compact", 
    "shadow-xl hover:shadow-2xl",
    ["card", "bg-base-100"]
)
print(f"Test 3 - Multiple sources: {result3}")

# Test with empty inputs
result4 = deduplicate_classes(None, [], "")
print(f"Test 4 - Empty inputs: '{result4}'")

# Test with complex responsive classes
result5 = deduplicate_classes(
    "text-sm md:text-base lg:text-lg",
    ["text-sm", "font-bold"],
    "hover:text-primary lg:text-lg"
)
print(f"Test 5 - Responsive classes: {result5}")

Testing deduplicate_classes function:
Test 1 - Duplicates: btn btn-lg btn-primary
Test 2 - With None: hidden lg:block md:block
Test 3 - Multiple sources: bg-base-100 card card-compact hover:shadow-2xl shadow-xl
Test 4 - Empty inputs: ''
Test 5 - Responsive classes: font-bold hover:text-primary lg:text-lg md:text-base text-sm


## Size Support Mixin

Mixin for components that support size modifiers.

In [ ]:
#| export
@dataclass 
class HasSize(CSSContributor):
    """Mixin for components that support size modifiers.
    
    This mixin provides size support for daisyUI components,
    including responsive size variations.
    
    Note: This mixin expects to be used with a class that has a
    component_class() method (like DaisyComponent).
    """
    
    # Size properties
    size: Optional[Union[DaisySize, str]] = None
    responsive_size: Optional[Dict[str, str]] = None  # e.g., {"md": "lg", "lg": "xl"}
    
    def get_css_classes(
        self
    ) -> List[str]:  # List of CSS class strings for size modifiers
        """Get size-related CSS classes.
        
        Returns:
            List of CSS class strings for size modifiers
        """
        classes: List[str] = []
        
        # Only add size classes if we have a component_class method
        if hasattr(self, 'component_class') and callable(self.component_class):
            try:
                base_class = self.component_class()
                
                # Add size modifier
                if self.size:
                    size_val = self.size.value if isinstance(self.size, DaisySize) else self.size
                    classes.append(f"{base_class}-{size_val}")
                
                # Add responsive size modifiers  
                if self.responsive_size:
                    for breakpoint, size in self.responsive_size.items():
                        classes.append(f"{breakpoint}:{base_class}-{size}")
            except:
                # If component_class raises an error, skip size classes
                pass
                
        return classes

## Base Component Class

The fundamental class that all daisyUI components inherit from.

In [ ]:
#| export
@dataclass
class DaisyComponent(ColorMixin, ComponentProtocol):
    """Base class for all daisyUI components.
    
    This class provides the foundation for building daisyUI components with:
    - Type-safe semantic color support with automatic content colors
    - Custom class and attribute support
    - Integration with cjm-tailwind-utils for additional styling
    - Responsive modifier support
    - Full implementation of ComponentProtocol interface
    """
    
    # HTML attributes
    id: Optional[str] = None
    cls: Optional[str] = None  # Additional custom classes
    attrs: Dict[str, Any] = field(default_factory=dict)
    
    # Responsive modifiers
    responsive_hide: Optional[List[str]] = None  # Breakpoints to hide at
    responsive_show: Optional[List[str]] = None  # Breakpoints to show at
    
    # Additional Tailwind customization
    tw_padding: Optional[Union[int, str]] = None
    tw_margin: Optional[Union[int, str]] = None
    tw_utilities: Optional[List[str]] = None  # Raw Tailwind utilities
    
    def component_class(
        self
    ) -> str:  # The base component class name (e.g., 'btn', 'card')
        """Return the base component class name (e.g., 'btn', 'card').
        
        Subclasses must implement this method.
        """
        raise NotImplementedError("Subclasses must implement component_class()")
    
    def modifier_classes(
        self
    ) -> List[str]:  # List of modifier CSS classes
        """Return all modifier classes for this component."""
        # Subclasses should override this to add their specific modifiers
        return []
    
    def build_classes(
        self
    ) -> str:  # Space-separated string of all CSS classes
        """Build complete class string with deduplication."""
        # Collect all classes from different sources
        class_sources = []
        
        # Add component classes
        class_sources.append(self.component_class())
        class_sources.extend(self.modifier_classes())
        
        # Add padding/margin
        tb = TailwindBuilder()
        if self.tw_padding is not None:
            tb.p(self.tw_padding)
        if self.tw_margin is not None:
            tb.m(self.tw_margin)
        
        # Get padding/margin classes from TailwindBuilder
        if tb.build():
            class_sources.extend(ensure_list(tb.build()))
            
        # Add responsive visibility
        responsive_classes = []
        if self.responsive_hide:
            for bp in self.responsive_hide:
                responsive_classes.append(f"{bp}:hidden")
        if self.responsive_show:
            for bp in self.responsive_show:
                responsive_classes.append(f"{bp}:block")
        class_sources.extend(responsive_classes)
                
        # Add raw utilities
        if self.tw_utilities:
            class_sources.extend(self.tw_utilities)
            
        # Collect CSS classes from all mixins that implement get_css_classes
        # Track which actual methods we've called to avoid duplicates
        called_methods = set()
        
        # Check each class in the MRO for get_css_classes method
        for cls in type(self).__mro__:
            if hasattr(cls, 'get_css_classes'):
                method = getattr(cls, 'get_css_classes')
                # Check if this is a unique method (not inherited)
                method_id = id(method.__func__ if hasattr(method, '__func__') else method)
                if callable(method) and method_id not in called_methods:
                    try:
                        result = method(self)
                        if result:
                            class_sources.extend(result)
                        called_methods.add(method_id)
                    except:
                        pass
        
        # Add custom classes last
        if self.cls:
            class_sources.extend(ensure_list(self.cls))
            
        # Use the deduplicate_classes function to build the final class string
        return deduplicate_classes(*class_sources)
    
    def render_attrs(
        self
    ) -> Dict[str, Any]:  # Dictionary of HTML attributes
        """Build all HTML attributes for rendering."""
        attrs: Dict[str, Any] = {**self.attrs}
        attrs["class"] = self.build_classes()
        
        if self.id:
            attrs["id"] = self.id
            
        return attrs
        
    def with_utilities(
        self,
        *utilities: str # Tailwind utility classes to add
    ) -> 'DaisyComponent':  # Self for method chaining
        """Add Tailwind utilities and return self for chaining."""
        if self.tw_utilities is None:
            self.tw_utilities = []
        self.tw_utilities.extend(utilities)
        return self
    
    def with_semantic_colors(
        self,
        bg: Optional[Union[SemanticColor, str]] = None, # Background color
        text: Optional[Union[SemanticColor, str]] = None, # Text color (auto-selected if None and auto_content=True)
        border: Optional[Union[SemanticColor, str]] = None, # Border color
        auto_content: bool = True  # Automatically select appropriate text color for background
    ) -> 'DaisyComponent':  # Self for method chaining
        """Apply semantic colors with automatic content color selection."""
        classes = ensure_list(apply_semantic_colors(bg, text, border, auto_content))
        return self.with_utilities(*classes)

## Semantic Color Integration

DaisyComponent now inherits from ColorMixin, providing powerful semantic color support:

- Use `SemanticColor` enum for type-safe color selection
- Automatic content color selection for accessibility
- Methods like `with_brand_colors()` and `with_state_colors()` for quick styling
- Full integration with daisyUI's theme system

In [ ]:
#| export
@dataclass
class ValidatedDaisyComponent(DaisyComponent):
    """Extended base class with component type validation.
    
    This optional base class adds validation to ensure component classes
    match known daisyUI component types.
    """
    
    # Component type for validation
    component_type: Optional[DaisyComponentType] = None
    
    def component_class(
        self
    ) -> str:  # The base component class name with validation
        """Return the base component class name with validation.
        
        If component_type is set, returns its value.
        Otherwise falls back to the standard implementation.
        """
        if self.component_type:
            return self.component_type.value
        return super().component_class()
    
    def validate_component_type(
        self
    ) -> None: # Validates component type if specified
        """Validate component type if specified.
        
        This should be called by subclasses after they've set up
        their component_type and implemented component_class.
        """
        if self.component_type:
            # For validation, temporarily clear component_type to get the actual class
            temp_type = self.component_type
            self.component_type = None
            try:
                actual = self.component_class()
            except NotImplementedError:
                # If component_class is not implemented yet, skip validation
                actual = None
            finally:
                self.component_type = temp_type
                
            if actual and actual != self.component_type.value:
                raise ValueError(
                    f"Component class mismatch: expected '{self.component_type.value}' "
                    f"but got '{actual}' for {self.component_type.name}"
                )

In [ ]:
# Example: Creating components with improved type safety
@dataclass
class ExampleComponent(DaisyComponent):
    def component_class(self) -> str:
        return "example"

# Example with validated component - simpler approach
@dataclass 
class ButtonComponent(ValidatedDaisyComponent):
    def __post_init__(self):
        # Set the component type
        self.component_type = DaisyComponentType.BUTTON
        # No need to call super().__post_init__() since we're using the enum value

# Debug the MRO for ExampleComponent
print("MRO for ExampleComponent:")
for i, cls in enumerate(ExampleComponent.__mro__):
    print(f"  {i}: {cls.__name__}")
    if hasattr(cls, 'get_css_classes'):
        print(f"     - has get_css_classes method")

print("\n" + "="*50 + "\n")

# Method 1: Using color methods from ColorMixin
example1 = ExampleComponent()
example1.with_brand_colors("primary")  # Sets bg-primary and text-primary-content
print("Method 1 - Brand colors:")
print(f"  Classes: {example1.build_classes()}")
print(f"  Color classes: {example1.get_css_classes()}")

# Method 2: Using semantic color helper
example2 = ExampleComponent()
example2.with_semantic_colors(
    bg=SemanticColor.SUCCESS,
    border=SemanticColor.SUCCESS,
    auto_content=True  # Automatically adds text-success-content
)
print("\nMethod 2 - Semantic colors:")
print(f"  Classes: {example2.build_classes()}")
print(f"  Utilities: {example2.tw_utilities}")

# Method 3: Using validated component
button = ButtonComponent()
button.with_brand_colors("secondary")
print("\nMethod 3 - Validated button component:")
print(f"  Component type: {button.component_type}")
print(f"  Component class: {button.component_class()}")
print(f"  Classes: {button.build_classes()}")
print(f"  Attrs: {button.render_attrs()}")

MRO for ExampleComponent:
  0: ExampleComponent
     - has get_css_classes method
  1: DaisyComponent
     - has get_css_classes method
  2: ColorMixin
     - has get_css_classes method
  3: CSSContributor
     - has get_css_classes method
  4: ComponentProtocol
  5: Protocol
  6: Generic
  7: object


Method 1 - Brand colors:
  Classes: bg-primary example text-primary-content
  Color classes: ['bg-primary', 'text-primary-content']

Method 2 - Semantic colors:
  Classes: bg-success border-success example text-success-content
  Utilities: ['bg-success', 'text-success-content', 'border-success']

Method 3 - Validated button component:
  Component type: DaisyComponentType.BUTTON
  Component class: btn
  Classes: bg-secondary btn text-secondary-content
  Attrs: {'class': 'bg-secondary btn text-secondary-content'}


In [ ]:
# Example: Using type aliases and utility functions
@dataclass
class CardComponent(DaisyComponent, HasSize):
    """Example card component demonstrating type alias usage."""
    
    # Using proper type aliases
    card_attrs: Dict[str, Any] = field(default_factory=dict)
    card_classes: List[str] = field(default_factory=list)
    
    def component_class(self) -> str:
        return DaisyComponentType.CARD.value
    
    def modifier_classes(self) -> List[str]:
        """Return card-specific modifiers."""
        modifiers: List[str] = []
        
        # Add any card-specific classes
        modifiers.extend(self.card_classes)
        
        return modifiers

# Create a card with various features
card = CardComponent(
    size=DaisySize.LG,
    responsive_size={"md": "xl", "lg": "2xl"},
    cls="shadow-xl hover:shadow-2xl",  # Will be split using ensure_list
    attrs={"data-theme": "cupcake"},
    card_attrs={"role": "article", "aria-label": "Product card"}
)

# Apply semantic colors
card.with_semantic_colors(
    bg=SemanticColor.BASE_200,
    text=SemanticColor.BASE_CONTENT
)

print("Card component demonstration:")
print(f"  Component class: {card.component_class()}")

# Test HasSize directly through the proper MRO
print("\nHasSize mixin contribution:")
# Create a temporary instance to see what HasSize contributes
size_mixin = HasSize(size=DaisySize.LG, responsive_size={"md": "xl", "lg": "2xl"})
# Manually add component_class for testing
size_mixin.component_class = lambda: "card"
print(f"  Size classes from mixin: {size_mixin.get_css_classes()}")
print(f"  Size classes in full build: {[c for c in card.build_classes().split() if 'card-' in c and c != 'card']}")

print(f"\n  All classes: {card.build_classes()}")
print(f"  Render attrs: {card.render_attrs()}")

# Demonstrate ensure_list utility
print("\nUsing ensure_list utility:")
single_class = "btn-primary"
multiple_classes = "btn btn-primary btn-lg"
class_list = ["btn", "btn-secondary"]

print(f"  Single: {ensure_list(single_class)}")
print(f"  Multiple: {ensure_list(multiple_classes)}")
print(f"  List: {ensure_list(class_list)}")

Card component demonstration:
  Component class: card

HasSize mixin contribution:
  Size classes from mixin: ['card-lg', 'md:card-xl', 'lg:card-2xl']
  Size classes in full build: ['card-lg', 'lg:card-2xl', 'md:card-xl']

  All classes: bg-base-200 card card-lg hover:shadow-2xl lg:card-2xl md:card-xl shadow-xl text-base-content
  Render attrs: {'data-theme': 'cupcake', 'class': 'bg-base-200 card card-lg hover:shadow-2xl lg:card-2xl md:card-xl shadow-xl text-base-content'}

Using ensure_list utility:
  Single: ['btn-primary']
  Multiple: ['btn', 'btn-primary', 'btn-lg']
  List: ['btn', 'btn-secondary']


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()